Some $\LaTeX$ abbreviations:
$$
\newcommand{\pd}[2]{\frac{\partial #1}{\partial #2}}
$$

In [1]:
%pylab inline
%config InlineBackend.figure_format = 'retina'

Populating the interactive namespace from numpy and matplotlib


In [2]:
import astropy.cosmology as cosmo
from astropy.cosmology import Planck15
import astropy.units as u
import h5py

In [3]:
def dmu_dz(mu):
    z, dl = mu
    dH = Planck15.hubble_distance.to(u.Gpc).value
    return np.array([1, dl/(1+z) + (1+z)*dH/Planck15.efunc(z)])

def dmu_dH0(mu):
    z, dl = mu
    return np.array([0.0, -dl/Planck15.H0.to(u.km/u.s/u.Mpc).value])

def dmu_dOm(mu):
    z, dl = mu
    
    zs = linspace(0, z, 100)
    ints = Planck15.hubble_distance.to(u.Gpc).value*(-((1+zs)**3-1)/(2*Planck15.efunc(zs)**3))
    
    return np.array([0.0, trapz(ints, zs)])

def dmu_dw(mu):
    z, dl = mu
    
    zs = linspace(0, z, 100)
    ints = Planck15.hubble_distance.to(u.Gpc).value*(-3)*(1-Planck15.Om0)*log(1+zs)/(2*Planck15.efunc(zs)**3)
    
    return np.array([0.0, trapz(ints, zs)])

def dmu_dwa(mu):
    z, dl = mu
    
    zs = linspace(0, z, 100)
    ints = Planck15.hubble_distance.to(u.Gpc).value*(-3)*(1-Planck15.Om0)*((1+zs)*log(1+zs) - zs)/(2*Planck15.efunc(zs)**3)
    
    return np.array([0.0, trapz(ints, zs)])

def fisher(z, dl, sigma_z, sigma_dl):
    mu = np.array([z, dl])
    
    Sigma = np.diag([1.0/sigma_z**2, 1.0/sigma_dl**2])
    
    dmu = np.array([dmu_dz(mu), dmu_dH0(mu), dmu_dOm(mu), dmu_dw(mu), dmu_dwa(mu)])
        
    F = np.dot(dmu, np.dot(Sigma, dmu.T))
    
    return F

In [4]:
with h5py.File('observations.h5', 'r') as f:
    m1true = array(f['m1s'])
    ztrue = array(f['zs'])
    dltrue = Planck15.luminosity_distance(ztrue).to(u.Gpc).value
    
    sigma_log_m1 = std(log(array(f['posteriors/m1det'])), axis=1)
    sigma_log_m2 = std(log(array(f['posteriors/m2det'])), axis=1)
    sigma_dl = std(array(f['posteriors/dl']), axis=1)

In [5]:
Fs = []
for z, dl, slm1, slm2, sd in zip(ztrue, dltrue, sigma_log_m1, sigma_log_m2, sigma_dl):
    slm = 1.0/sqrt(1.0/slm1**2 + 1.0/slm2**2)
    Fs.append(fisher(z, dl, slm, sd))
Fs = array(Fs)

In [7]:
F = np.sum(Fs, axis=0)
F_constr = F + np.diag([0.0, 1.0/(0.01*Planck15.H0.to(u.km/u.s/u.Mpc).value)**2, 1.0/(Planck15.Om0*0.01)**2, 0.0, 0.0])

In [8]:
C = np.linalg.inv(F)
C_constr = np.linalg.inv(F_constr)

In [9]:
sqrt(diag(C)), sqrt(diag(C_constr))

(array([7.25979979e-04, 1.64247423e+00, 8.69420538e-01, 9.22190572e-01,
        5.36941905e+00]),
 array([5.45387658e-04, 5.68556651e-01, 3.07496090e-03, 1.12304851e-01,
        3.30600733e-01]))

In [11]:
evals, evecs = np.linalg.eigh(C_constr[3:,3:])
evals, evecs

(array([0.00746145, 0.11444777]), array([[-0.97563022, -0.21942122],
        [-0.21942122,  0.97563022]]))

In [12]:
sqrt(evals[0]), evecs[1,0]/evecs[0,0]

(0.08637969279809281, 0.22490203038905476)

In [13]:
ap = 1-evecs[1,0]/evecs[0,0]
zp = 1/ap-1
zp, ap

(0.2901594884862655, 0.7750979696109452)